<div align='center'>

# Practica 2

<img src='https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExbXVoOXh6bGtwend4cW5majMwZHRiYzh6b2loZGNseGZmajMxbmhsNiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/aMwdfGSyeYjUKY6vjf/giphy.gif'>

</div>

---

## Ejercicio 1)
¿En el dataset del ejercicio 1 de la práctica 1 indique para cada Job, si se vería beneficiado por una función combiner?

En caso afirmativo,

¿cuál es la implementación de dicha función? ¿Qué datos recibe cada reduce, al utilizar la función combiner?

---

## Ejercicio 2)
Implemente una función combiner para el problema del WordCount.

In [1]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "./WordCount/input/"
outputDir = "./WordCount/ejercicio_2/"

def fmap(key, value, context):
    words = value.split()
    for w in words:
        context.write(w, 1)
        
def fred(key, values, context):
    c=0
    for v in values:
        c=c+1
    context.write(key, c)

job = Job(inputDir, outputDir, fmap, fred)
job.setCombiner(fred)
success = job.waitForCompletion()
print(success)

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] EL -> 1
[Map] PAPAGAYO -> 1
[Map] DE -> 1
[Map] HUICHILOBOS -> 1
[Map] Repentinamente -> 1
[Map] oí -> 1
[Map] una -> 1
[Map] exclamación -> 1
[Map] de -> 1
[Map] sorpresa -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: !ah! -> [1]
[Reduce] Clave recibida: "A" -> [1]
[Reduce] Clave recibida: "B" -> [1]
[Reduce] Clave recibida: "Dr. -> [1]
[Reduce] Clave recibida: "El -> [1]
[Reduce] Clave recibida: "purgar", -> [1]
[Reduce] Clave recibida: (aquÃ­ -> [1]
[Reduce] Clave recibida: (con -> [1]
[Reduce] Clave recibida: (naturalmente) -> [1]
[Reduce] Clave recibida: * -> [11, 1, 1, 1, 1]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
!ah!	1
"A"	1
"B"	1
"Dr.	1
"El	1
"purgar",	1
(aquÃ­	1
(con	1
(naturalmente)	1
*	5
... [más resultados omitidos]
True


---

## Ejercicio 3)
Implemente un job MapReduce para calcular el máximo, mínimo, promedio y desvío stándard de las ocurrencias de todas las palabras del dataset Libros.

#### Cuenta ocurrencias de cada palabra.


In [2]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "./Libros/input"
wcDir = "./Libros/ej3_wc/"

def fmap_wc(key, value, context):
    # separar por espacios
    for palabra in value.strip().split():
        palabra = palabra.lower().strip(".,;:¡!¿?\"()[]")
        if palabra:
            context.write(palabra, 1)

def fred_wc(key, values, context):
    total = 0
    for v in values:
        total += v
    context.write(key, total)

job_wc = Job(inputDir, wcDir, fmap_wc, fred_wc)
job_wc.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] tristis -> 1
[Map] imago -> 1
[Map] --¡rafael -> 1
[Map] exclamé -> 1
[Map] pero -> 1
[Map] él -> 1
[Map] me -> 1
[Map] interrumpió -> 1
[Map] diciendo -> 1
[Map] el -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: * -> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, '...']
[Reduce] Clave recibida: --"estimado -> [1]
[Reduce] Clave recibida: --amigo -> [1, 1]
[Reduce] Clave recibida: --angustias -> [1]
[Reduce] Clave recibida: --aquí -> [1]
[Reduce] Clave recibida: --aun -> [1]
[Reduce] Clave recibida: --buenas -> [1, 1]
[Reduce] Clave recibida: --buenos -> [1, 1]
[Reduce] Clave recibida: --comprendo -> [1]
[Reduce] Clave recibida: --doctor -> [1]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
*	15
--"estimado	1
--amigo	2
--angustias	1
--aquí	1
--aun	1
--buenas	2
--buenos	2
--comprendo	1
--doctor	1
... [más resultados omitidos]


True

#### Job de estadísticas (min, max, promedio)

In [5]:
statsDir = "./Libros/ej3_stats/"

def fmap_stats(key, value, context):
    if value.strip():  # saltear líneas vacías
        ocurr = int(value.strip())
        context.write("min", ocurr)
        context.write("max", ocurr)
        context.write("avg", ocurr)


def fred_stats(key, values, context):
    if key == "min":
        context.write("min", min(values))
    elif key == "max":
        context.write("max", max(values))
    elif key == "avg":
        total, n = 0, 0
        for v in values:
            total += v
            n += 1
        context.write("promedio", total / n)

job_stats = Job(wcDir, statsDir, fmap_stats, fred_stats)
job_stats.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] min -> 15
[Map] max -> 15
[Map] avg -> 15
[Map] min -> 1
[Map] max -> 1
[Map] avg -> 1
[Map] min -> 1
[Map] max -> 1
[Map] avg -> 1
[Map] min -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: avg -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[Reduce] Clave recibida: max -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[Reduce] Clave recibida: min -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
promedio	3.4946911196911197
max	839
min	1


True

#### Job de desvío estándar

In [8]:
desvioDir = "./Libros/ej3_desvio/"

import math

def fmap_desvio(key, value, context):
    if value.strip():  # saltear líneas vacías
        ocurr = int(value.strip())
        avg = context["avg"]
        context.write("var", (ocurr - avg) ** 2)

def fred_desvio(key, values, context):
    total, n = 0, 0
    for v in values:
        total += v
        n += 1
    # varianza muestral
    var = total / (n - 1)
    context.write("desvio", math.sqrt(var))

# leer promedio del job_stats
with open(statsDir + "output.txt", "r") as f:
    lines = f.readlines()

prom_line = [l for l in lines if l.startswith("promedio")][0]
promedio = float(prom_line.strip().split('\t')[1])

params = {"avg": promedio}
job_desvio = Job(wcDir, desvioDir, fmap_desvio, fred_desvio)
job_desvio.setParams(params)
job_desvio.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] var -> 132.3721324313144
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 0.24471930390125368
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: var -> [132.3721324313144, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 0.24471930390125368, '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
desvio	22.917022729315015


True

---

## Ejercicio 4
Utilice el dataset Libros para implementar una aplicación MapReduce que devuelva como salida todos los párrafos que tienen una longitud mayor al promedio.


---

## Ejercicio 5

El dataset website tiene información sobre el tiempo de permanencia de sus usuarios
en cada una de las páginas del sitio. El formato de los datos del dataset es:
<id_user, id_page, time>
Implemente una aplicación MapReduce, utilizando combiners en los casos que
considere necesario, que calcule

> Indique como queda el DAG del proceso completo (las tres consultas)



---

#### a. La página más visitada (la página en la que más tiempo permaneció) para cada usuario

---

#### b. El usuario que más páginas distintas visitó

---

#### c. La página más visitada (en cuanto a cantidad de visitas, sin importar el tiempo de permanencia) por todos los usuarios.

---

## 6) Cómo plantearía una solución MapReduce a los siguientes algoritmos secuenciales:

i. entrada

```
textos: array [1..N] of string (dataset libros)
```
ii. algoritmo

```python
a={}; b={}; N = len(textos)
for l in textos:
    words = l.split()
    for w in words:
        a[w] = a[w]+1

for w in a.keys():
    for l in lines:
        words = l.split()
        if w in words:
            b[w]=b[w]+1
for k in a.keys():
    print(k + " = " + str(a[w] * (N / b[w])))
```

b.
i. entrada
```
datos: array [1..N] of /<int1, int2, ..., intM> (todos los valores están dentro de un rango de valores conocido, para poder usarlos como índices del tensor)
```
ii. algoritmo

```python
for t in datos:
    v = t.split("\t")
    c = v[-1]
        for a in range(len(v)-1):
        x= v[a]
        m[a][x][c] = m[a][x][c] + 1

max=[[0,0,0], [0,0,0]]
for x in range(len(m)):
    for y in range(len(m[0])):
        for z in range(2):
            if(m[x][y][z] > max[z][0]):
                max[z][0] = m[x][y][z]
                max[z][1]=x
                max[z][2]=y

for z in range(2):
    print(z +";" + max[z][1] +";" + max[z][2])
```